# example

--|steel|oil|chip|bean|clothes
--|--|--|--|--|--
USA|1|1|1|1|0
China|1|0|0|1|1
Viet Nam|0|0|0|1|1

In [128]:
import numpy as np


def RCA(Xcp):
    """
    Xcp is a numpy.matrix type
    --|steel|oil|chip|bean|clothes
    --|--|--|--|--|--
    USA|1|1|1|1|0
    China|1|0|0|1|1
    Viet Nam|0|0|0|1|1
    """
    if type(Xcp) is not np.matrix:
        if type(Xcp) is np.array or list:
            Xcp=np.matrix(Xcp)
        else:
            raise ValueError('Xcp must be matrix')
    else:
        pass
    B=Xcp.sum(1)*Xcp.sum(0) 
    Sum=Xcp.sum()
    return Xcp*Sum/B
    
def fliterRCA(R):
    M=R>1
    return M.astype(float)



In [296]:
# example data
product_name=["steel","oil","chip","bean","clothes"]
country_name=["USA","China","Viet Nam"]
data=[[1,1,1,1,0],[1,0,0,1,1],[0,0,0,1,1]]
data=np.matrix(data)
R=RCA(data)
M=fliterRCA(R)

In [335]:
D=np.diag(M.sum(1).T.tolist()[0])
U=np.diag(M.sum(0).tolist()[0])
mcp1 = np.linalg.inv(D)*M
mcp2 = M*np.linalg.inv(U)
Mcc = mcp1 @ mcp2.T
Mpp = mcp2.T @ mcp1

In [351]:
eigvals, eigvecs = np.linalg.eig(Mpp)
eigvecs = np.real(eigvecs)
# Get eigenvector corresponding to second largest eigenvalue
eig_index = eigvals.argsort()[-2]
kp = eigvecs[:, eig_index]
kc = mcp1 @ kp
s1 = np.sign(np.corrcoef(M.sum(1).reshape(-1), kc.reshape(-1))[0, 1])
eci = s1 * kc
pci = s1 * kp

In [352]:
eci

matrix([[ 0.38230021],
        [-0.08766502],
        [-0.48578529]])

In [353]:
pci

matrix([[ 0.18527832],
        [ 0.48081115],
        [ 0.48081115],
        [-0.61096222],
        [-0.36060836]])

In [349]:
s1 = np.sign(np.corrcoef(M.sum(1).reshape(-1), kc.reshape(-1))[0, 1])

In [350]:
s1

1.0